In [123]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [237]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet").head(20000)
rec_system_copy = rec_system.copy()

In [236]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],None,[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],None,None,[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],None,"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],None,"[Procirep, Media Programme of the European Com...",[]


In [151]:
rec_system_copy.isna().sum()

title                0
overview           133
name_genre         461
actors_names       561
director_names     158
tagline           8294
company           3453
collection           0
dtype: int64

In [238]:
rec_system.fillna({"overview":"[]",
                   "name_genre":"[]",
                   "actors_names":"[]",
                   "director_names":"[]",
                   "tagline":"[]",
                   "company":"[]",
                   },inplace=True)

* Transformo la columna overview a una lista con la funcion `split`

In [127]:
#NO
# rec_system["overview"] = rec_system["overview"].apply(lambda x: x.split())
# rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())

In [154]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,So goes to the U.S. to open a martial arts sch...,"[Action, Western]","[Xiong Xin-Xin, Johnny Koo, Freddy Joe, T. J. ...","[Lau Kar-Wing, Sammo Hung]",[],"[Win's Entertainment Ltd., China Star Entertai...",Once Upon a Time in China Collection
19996,contract killer,"Financially troubled, a newbie hitman reluctan...","[Comedy, Action]","[Fukui Akihiro, Mitsuru Tokashiki, Hideri Meik...",[Wei Tung],[],"[Win's Entertainment Ltd., Flea Market, China ...",[]
19997,the executor,Chow Yun-Fat and Danny Lee star in this police...,"[Crime, Action, Drama]","[Kwan Hoi-San, Lily Li Li-Li, Paul Chun Pui, C...","[Chuan Chen, Chik Ngai-Hung, Yi-Hsiung Chi]",[],"[Golden Tripod Film Co., Hong Kong]",[]
19998,never say... never!,"Paris, 1830. Valentin loses himself in alcohol...",[Comedy],"[Jean-Luc Porraz, Jacques Herlin, Marie-France...",[Eric Civanyan],[],"[SND, TPS Star, TF1 Films Production]",[]


In [239]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i)
    return valores

In [155]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,So goes to the U.S. to open a martial arts sch...,"[Action, Western]","[Xiong Xin-Xin, Johnny Koo, Freddy Joe, T. J. ...","[Lau Kar-Wing, Sammo Hung]",[],"[Win's Entertainment Ltd., China Star Entertai...",Once Upon a Time in China Collection
19996,contract killer,"Financially troubled, a newbie hitman reluctan...","[Comedy, Action]","[Fukui Akihiro, Mitsuru Tokashiki, Hideri Meik...",[Wei Tung],[],"[Win's Entertainment Ltd., Flea Market, China ...",[]
19997,the executor,Chow Yun-Fat and Danny Lee star in this police...,"[Crime, Action, Drama]","[Kwan Hoi-San, Lily Li Li-Li, Paul Chun Pui, C...","[Chuan Chen, Chik Ngai-Hung, Yi-Hsiung Chi]",[],"[Golden Tripod Film Co., Hong Kong]",[]
19998,never say... never!,"Paris, 1830. Valentin loses himself in alcohol...",[Comedy],"[Jean-Luc Porraz, Jacques Herlin, Marie-France...",[Eric Civanyan],[],"[SND, TPS Star, TF1 Films Production]",[]


In [240]:
# Con parquet, los tipos list pasan a ser ndarray, con este metodo los volvemos lista
rec_system["name_genre"]=rec_system["name_genre"].apply(collapse)
rec_system["actors_names"]=rec_system["actors_names"].apply(collapse)
rec_system["director_names"]=rec_system["director_names"].apply(collapse)
rec_system["company"]=rec_system["company"].apply(collapse)
# rec_system["tagline"]=rec_system["tagline"].apply(collapse)

In [242]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],[],"[[, ]]",[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],[],"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],[],"[Procirep, Media Programme of the European Com...",[]


In [243]:
rec_system["name_genre"] = rec_system["name_genre"].apply(lambda x: ",".join(x))
rec_system["actors_names"] = rec_system["actors_names"].apply(lambda x: ",".join(x))
rec_system["director_names"] = rec_system["director_names"].apply(lambda x: ",".join(x))
rec_system["company"] = rec_system["company"].apply(lambda x: ",".join(x))

In [254]:
rec_system_copy

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Comedy,Animation,Family","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"Fantasy,Adventure,Family","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"Romance,Comedy","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy,Drama,Romance","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","Romance,Drama","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],[],"[[, ]]",[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,Drama,"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"Action,Drama,History","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],[],"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,Drama,"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],[],"[Procirep, Media Programme of the European Com...",[]


In [256]:
for i in rec_system_copy["company"]:
    print(i)
    break

['Pixar Animation Studios']


In [251]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet").head(20000)
rec_system_copy = rec_system.copy()

rec_system_copy.fillna({"overview":"[]",
                   "name_genre":"[]",
                   "actors_names":"[]",
                   "director_names":"[]",
                   "tagline":"[]",
                   "company":"[]",
                   },inplace=True)

def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i)
    return valores

print(rec_system_copy)

rec_system_copy["company"]=rec_system_copy["company"].apply(collapse)
rec_system_copy["name_genre"]=rec_system_copy["name_genre"].apply(collapse)
# rec_system_copy["actors_names"]=rec_system_copy["actors_names"].apply(collapse)
# rec_system_copy["director_names"]=rec_system_copy["director_names"].apply(collapse)

rec_system_copy["name_genre"] = rec_system_copy["name_genre"].apply(lambda x: ",".join(x))

rec_system_copy["tags"] = rec_system_copy["overview"] + " " + rec_system_copy["name_genre"]  + " " + rec_system_copy["tagline"] + rec_system_copy["company"] #+ rec_system_copy["collection"]

                             title  \
0                        Toy Story   
1                          Jumanji   
2                 Grumpier Old Men   
3                Waiting to Exhale   
4      Father of the Bride Part II   
...                            ...   
19995           After Fall, Winter   
19996       Violeta Went to Heaven   
19997                    Rebellion   
19998                   Versailles   
19999              Two in the Wave   

                                                overview  \
0      Led by Woody, Andy's toys live happily in his ...   
1      When siblings Judy and Peter discover an encha...   
2      A family wedding reignites the ancient feud be...   
3      Cheated on, mistreated and stepped on, the wom...   
4      Just when George Banks has recovered from his ...   
...                                                  ...   
19995  WINTER is a dangerous, sexy, poignant and at t...   
19996  A portrait of famed Chilean singer and folklor...   
199

TypeError: can only concatenate str (not "list") to str

In [244]:
rec_system["tags"] =  rec_system["overview"] + " " + rec_system["name_genre"]  + " " + rec_system["tagline"] + rec_system["company"] + rec_system["collection"] #+ rec_system["actors_names"] + rec_system["director_names"]+

In [202]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,tags
0,toy story,"Led by Woody, Andy's toys live happily in his ...","Comedy,Animation,Family","Sarah Freeman,Tim Allen,Annie Potts,Penn Jille...",John Lasseter,[],Pixar Animation Studios,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ..."
1,jumanji,When siblings Judy and Peter discover an encha...,"Fantasy,Adventure,Family","Malcolm Stewart,David Alan Grier,Bradley Pierc...",Joe Johnston,Roll the dice and unleash the excitement!,"Interscope Communications,TriStar Pictures,Tei...",[],When siblings Judy and Peter discover an encha...
2,grumpier old men,A family wedding reignites the ancient feud be...,"Romance,Comedy","Jack Lemmon,Burgess Meredith,Walter Matthau,An...",Howard Deutch,Still Yelling. Still Fighting. Still Ready for...,"Warner Bros.,Lancaster Gate",Grumpy Old Men Collection,A family wedding reignites the ancient feud be...
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy,Drama,Romance","Angela Bassett,Mykelti Williamson,Lela Rochon,...",Forest Whitaker,Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],"Cheated on, mistreated and stepped on, the wom..."
4,father of the bride part ii,Just when George Banks has recovered from his ...,Comedy,"Kimberly Williams-Paisley,Lori Alan,BD Wong,Eu...",Charles Shyer,Just When His World Is Back To Normal... He's ...,"Sandollar Productions,Touchstone Pictures",Father of the Bride Collection,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,So goes to the U.S. to open a martial arts sch...,"Action,Western","Xiong Xin-Xin,Johnny Koo,Freddy Joe,T. J. Stor...","Lau Kar-Wing,Sammo Hung",[],"Win's Entertainment Ltd.,China Star Entertainment",Once Upon a Time in China Collection,So goes to the U.S. to open a martial arts sch...
19996,contract killer,"Financially troubled, a newbie hitman reluctan...","Comedy,Action","Fukui Akihiro,Mitsuru Tokashiki,Hideri Meiken,...",Wei Tung,[],"Win's Entertainment Ltd.,Flea Market,China Sta...",[],"Financially troubled, a newbie hitman reluctan..."
19997,the executor,Chow Yun-Fat and Danny Lee star in this police...,"Crime,Action,Drama","Kwan Hoi-San,Lily Li Li-Li,Paul Chun Pui,Chow ...","Chuan Chen,Chik Ngai-Hung,Yi-Hsiung Chi",[],"Golden Tripod Film Co., Hong Kong",[],Chow Yun-Fat and Danny Lee star in this police...
19998,never say... never!,"Paris, 1830. Valentin loses himself in alcohol...",Comedy,"Jean-Luc Porraz,Jacques Herlin,Marie-France Sa...",Eric Civanyan,[],"SND,TPS Star,TF1 Films Production",[],"Paris, 1830. Valentin loses himself in alcohol..."


In [203]:
rec_system["tags"] = rec_system["tags"].apply(lambda x: "".join(x))

In [204]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,tags
0,toy story,"Led by Woody, Andy's toys live happily in his ...","Comedy,Animation,Family","Sarah Freeman,Tim Allen,Annie Potts,Penn Jille...",John Lasseter,[],Pixar Animation Studios,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ..."
1,jumanji,When siblings Judy and Peter discover an encha...,"Fantasy,Adventure,Family","Malcolm Stewart,David Alan Grier,Bradley Pierc...",Joe Johnston,Roll the dice and unleash the excitement!,"Interscope Communications,TriStar Pictures,Tei...",[],When siblings Judy and Peter discover an encha...
2,grumpier old men,A family wedding reignites the ancient feud be...,"Romance,Comedy","Jack Lemmon,Burgess Meredith,Walter Matthau,An...",Howard Deutch,Still Yelling. Still Fighting. Still Ready for...,"Warner Bros.,Lancaster Gate",Grumpy Old Men Collection,A family wedding reignites the ancient feud be...
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy,Drama,Romance","Angela Bassett,Mykelti Williamson,Lela Rochon,...",Forest Whitaker,Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],"Cheated on, mistreated and stepped on, the wom..."
4,father of the bride part ii,Just when George Banks has recovered from his ...,Comedy,"Kimberly Williams-Paisley,Lori Alan,BD Wong,Eu...",Charles Shyer,Just When His World Is Back To Normal... He's ...,"Sandollar Productions,Touchstone Pictures",Father of the Bride Collection,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,So goes to the U.S. to open a martial arts sch...,"Action,Western","Xiong Xin-Xin,Johnny Koo,Freddy Joe,T. J. Stor...","Lau Kar-Wing,Sammo Hung",[],"Win's Entertainment Ltd.,China Star Entertainment",Once Upon a Time in China Collection,So goes to the U.S. to open a martial arts sch...
19996,contract killer,"Financially troubled, a newbie hitman reluctan...","Comedy,Action","Fukui Akihiro,Mitsuru Tokashiki,Hideri Meiken,...",Wei Tung,[],"Win's Entertainment Ltd.,Flea Market,China Sta...",[],"Financially troubled, a newbie hitman reluctan..."
19997,the executor,Chow Yun-Fat and Danny Lee star in this police...,"Crime,Action,Drama","Kwan Hoi-San,Lily Li Li-Li,Paul Chun Pui,Chow ...","Chuan Chen,Chik Ngai-Hung,Yi-Hsiung Chi",[],"Golden Tripod Film Co., Hong Kong",[],Chow Yun-Fat and Danny Lee star in this police...
19998,never say... never!,"Paris, 1830. Valentin loses himself in alcohol...",Comedy,"Jean-Luc Porraz,Jacques Herlin,Marie-France Sa...",Eric Civanyan,[],"SND,TPS Star,TF1 Films Production",[],"Paris, 1830. Valentin loses himself in alcohol..."


In [205]:
# max_features: maxima cantidad de palabras a vectorizar
# cv = CountVectorizer(max_features=5000, stop_words="english")
tv = TfidfVectorizer(max_features=5000, stop_words="english")

In [206]:
vector = tv.fit_transform(rec_system["title"]).toarray()

In [234]:
tv.vocabulary_ # Chequeo palabras que se repitieron

{'toy': 4512,
 'story': 4224,
 'old': 3036,
 'men': 2739,
 'waiting': 4754,
 'father': 1550,
 'bride': 587,
 'ii': 2139,
 'heat': 1986,
 'sabrina': 3753,
 'tom': 4479,
 'huck': 2096,
 'sudden': 4269,
 'death': 1156,
 'american': 174,
 'president': 3411,
 'dracula': 1308,
 'dead': 1147,
 'loving': 2580,
 'balto': 338,
 'nixon': 2982,
 'island': 2206,
 'casino': 743,
 'sense': 3864,
 'rooms': 3705,
 'ace': 94,
 'ventura': 4685,
 'nature': 2934,
 'calls': 684,
 'money': 2837,
 'train': 4526,
 'assassins': 263,
 'powder': 3374,
 'leaving': 2459,
 'las': 2431,
 'vegas': 4677,
 'othello': 3073,
 'persuasion': 3200,
 'city': 861,
 'lost': 2566,
 'children': 821,
 'shanghai': 3905,
 'dangerous': 1119,
 'minds': 2790,
 'monkeys': 2840,
 'wings': 4885,
 'courage': 1012,
 'babe': 309,
 'man': 2639,
 'walking': 4761,
 'sea': 3836,
 'time': 4463,
 'takes': 4339,
 'beloved': 423,
 'country': 1008,
 'richard': 3638,
 'iii': 2140,
 'presidents': 3413,
 'mortal': 2861,
 'kombat': 2392,
 'die': 1230,
 '

In [208]:
cosine_sim = linear_kernel(vector,vector)

In [209]:
indices = pd.Series(rec_system.index, index=rec_system["title"]).drop_duplicates()

In [210]:
# similarity = cosine_similarity(vector)

In [221]:
def recommend(movie,cosine_sim=cosine_sim):
    movie = movie.lower()
    index = indices[movie]
    sim_scores=list(enumerate(cosine_sim[index]))
    distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
    
    for i in distances[1:6]:
        # print(i[0])
        print(rec_system.iloc[i[0]].title)
recommend("")

jumanji
grumpier old men
waiting to exhale
father of the bride part ii
heat
